In [2]:
import sys, pickle
import sklearn.preprocessing as pre, scipy, numpy as np, matplotlib.pyplot as plt, glob, os, pyemma, subprocess
import pandas as pd, seaborn as sns, argparse
import ipywidgets 

#from temp_tf_load import *
sys.path.append('../../hde')

import warnings
warnings.filterwarnings('ignore')
from hde import HDE, analysis

In [5]:
# get temp and n slow modes determined by cross-val
srv_params = {'AT-all': [309, 5, 6],
              'GC-end': [317, 5, 5],
              'GC-core': [324, 4, 4],
              'GC-mix': [324, 2, 3]}

reduced = False

for seq in ['GC-mix']: #['AT-all', 'GC-end', 'GC-core', 'GC-mix']:

    [temp, hde_nsm, nMacroStates] = srv_params[seq]

    # lag times
    lag = 12
    srv_lag = 12
    srv_ep = 20

    # number of cluster centers and clustering stride
    nClusterCentres = 200
    stride = 1000


    # load traj data and hde path
    load_path = '/home/mikejones/scratch-midway2/srv/implicit/'
    
    if reduced:
        npy_equ =   f'dna_data/{seq}_dist_reduced_{temp}K_40-250000-55.npy'
        data = 1 / np.load(load_path+npy_equ)
    else:
        npy_equ =   f'dna_data/{seq}_dist_{temp}K_40-250000-190.npy'
        data = 1 / np.load(load_path+npy_equ)[:, :, :100]
    
    hde_name =  f"srv_out/sm-{hde_nsm}_k-0_lag-{srv_lag}_ep-{srv_ep}_{seq}{npy_equ.split(seq)[-1].replace('npy', 'pkl')}"

    # create a directory to aggregate msm data
    save_dir = f"MSMs/{npy_equ.split('/')[-1].replace('.npy', '')}"
    save_dir += f'_micro-{nClusterCentres}_macro-{nMacroStates}_lag-{lag}_srvlag-{srv_lag}/'
    subprocess.run(['mkdir', save_dir])
    subprocess.run(['cp', hde_name, save_dir+'hde.pkl'])

    # scales all features to the range 0 and 1
    scaler = pre.MinMaxScaler(feature_range=(0, 1))
    scaler.fit(np.concatenate(data))
    data_s = [scaler.transform(item) for item in data]
    print(np.shape(data_s))

    # transform coordinates based on loaded srv
    hde = pickle.load(open(load_path+hde_name, 'rb'))
    hdeOutput = [hde.transform(x) for x in data_s] 
    print(np.shape(hdeOutput))

    # compare with tica, use same dimensions and lag as srv for consistency
    tica = pyemma.coordinates.tica([traj for traj in data_s], lag=srv_lag, dim=hde_nsm)
    ticaOutput = tica.get_output()
    print(np.shape(ticaOutput))
    
    #save coord tranformations
    hde_coords = save_dir+'hde_coords.pkl'
    with open(hde_coords , 'wb') as f:
        pickle.dump(hdeOutput, f)
    f.close()

    tica_coords = save_dir+'tica_coords.pkl'
    with open(tica_coords, 'wb') as f:
        pickle.dump(ticaOutput, f)
    f.close()
    
    
    # cluster based on srv and tica dynamical representations
    cluster_hde = pyemma.coordinates.cluster_kmeans(hdeOutput, 
                        stride=stride, k=nClusterCentres, max_iter=50)

    cluster_tica = pyemma.coordinates.cluster_kmeans(ticaOutput, 
                        stride=stride, k=nClusterCentres, max_iter=50)

    # extract discrete trajectories along microstates found above
    dtraj_tica = cluster_tica.dtrajs
    dtraj_hde = cluster_hde.dtrajs

    #save cluster data
    cluster_hde_name = save_dir+f'hde_cluster.pkl'
    with open(cluster_hde_name, 'wb') as f:
        pickle.dump(cluster_hde, f)
    f.close()

    cluster_tica_name = save_dir+f'tica_cluster.pkl'
    with open(cluster_tica_name, 'wb') as f:
        pickle.dump(cluster_tica, f)
    f.close()

    #save cluster data
    dtraj_hde_name = save_dir+f'dtraj_hde.pkl'
    with open(dtraj_hde_name, 'wb') as f:
        pickle.dump(dtraj_hde, f)
    f.close()

    dtraj_tica_name = save_dir+f'dtraj_tica.pkl'
    with open(dtraj_tica_name, 'wb') as f:
        pickle.dump(dtraj_tica, f)
    f.close()
    
    msm_hde = pyemma.msm.bayesian_markov_model(dtraj_hde, lag)
    msm_tica = pyemma.msm.bayesian_markov_model(dtraj_tica, lag)

    # cluster each into nMacroStates macro clusters
    pcca_hde = msm_hde.pcca(nMacroStates)
    pcca_tica = msm_tica.pcca(nMacroStates)

    # save hde msm
    msm_hde_name = save_dir+f'msm_hde.pkl'
    with open(msm_hde_name, 'wb') as f:
        pickle.dump(msm_hde, f)
    f.close()

    # save tica msm
    msm_tica_name = save_dir+f'msm_tica.pkl'
    with open(msm_tica_name, 'wb') as f:
        pickle.dump(msm_tica, f)
    f.close()
    

(40, 250000, 100)
(40, 250000, 2)


(40, 250000, 2)
